# Context-Aware Recommendation Algorithm by PAPERDAA-2

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import preprocessing
from itertools import product as cartesian_product

# Training

## Program Arguments

In [2]:
LMAX = 10    # Maximum transitive path length
K = 50       # Nearest neighbors

## Preprocessing

In [3]:
df = pd.read_csv('training_dataset.csv')

df.tail(5)

,userid,itemid,rating,Time,Location,Companion
3432,1082,tt0413267,1,Weekend,Home,Partner
3433,1082,tt1637706,2,Weekend,Home,Partner
3434,1082,tt0343660,1,Weekend,Home,Partner
3435,1082,tt1133985,1,Weekend,Home,Partner
3436,1082,tt1099212,1,Weekend,Home,Partner


In [4]:
encoder = preprocessing.LabelEncoder()

user_item_context_encodings = []
user_item_context_reverse_encodings = []
maximum_rating = df[df.columns[2]].max()

encoded_df = df.copy()
# Encode userid, itemid, and contextual informations for item splitting
for column_index in range(len(df.columns)):
    
    # Column attribute is not rating
    if column_index != 2:
        
        # Fit encoder
        encoder.fit(df[df.columns[column_index]])
        encoded_df[df.columns[column_index]] = encoder.transform(
                df[df.columns[column_index]]
            )
    
    # Column is nor user or rating
    if column_index != 2:
            user_item_context_encodings.append(
                dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
            )
            user_item_context_reverse_encodings.append(
                dict(zip(encoder.transform(encoder.classes_), encoder.classes_))
            )

In [5]:
display(encoded_df)

,userid,itemid,rating,Time,Location,Companion
0,2,38,1,0,0,0
1,2,8,1,0,0,0
2,2,74,1,0,0,0
3,2,5,1,0,0,0
4,2,2,1,0,0,0
...,...,...,...,...,...,...
3432,65,35,1,1,1,2
3433,65,62,2,1,1,2
3434,65,25,1,1,1,2
3435,65,50,1,1,1,2


## Item Splitting

In [6]:
# Cartesian product all items and contexts

users = user_item_context_encodings[0].values()
items = user_item_context_encodings[1].values()
contexts = [
    context_trans.values() for context_trans in user_item_context_encodings[2:]
]

context_T = list(cartesian_product(items, *contexts))

In [7]:
# Generate new user-item matrix for new items
rating_matrix = np.zeros((len(users), len(context_T)), dtype=object)

In [8]:
for row in encoded_df.iterrows():
    data = tuple(row[1])
    user = data[0]
    item = data[1]
    rating = data[2]
    context_item = (item, *data[3:])
    
    index = context_T.index(context_item)
    
    rating_matrix[user][index] = int(rating)/int(maximum_rating)

In [9]:
display(rating_matrix)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0.2, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1.0, 0.8, 1.0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

## Graph Similarity Calculation (Item-Based)

In [10]:
rating_matrix_transposed = np.transpose(rating_matrix)

In [11]:
L = 2

WTW = np.matmul(rating_matrix_transposed, rating_matrix)
M = np.matmul(rating_matrix_transposed, rating_matrix)

while L != LMAX:
    M = np.matmul(WTW, M)
    L = L + 2

In [12]:
display(M)

array([[161362854.8928076, 163387168.910615, 271258083.63586247, ...,
        8830021.357861277, 47441837.88512668, 100245293.7101391],
       [163387168.91061497, 230350965.46172532, 282495567.9929451, ...,
        12059442.989648694, 56546089.31476934, 143676882.7843655],
       [271258083.63586223, 282495567.99294555, 495368058.0833022, ...,
        15629205.441586388, 83337451.74549294, 167637019.77834514],
       ...,
       [8830021.357861271, 12059442.989648692, 15629205.441586375, ...,
        704531.439828992, 3016398.0204855297, 7825859.198666341],
       [47441837.88512663, 56546089.31476941, 83337451.74549283, ...,
        3016398.0204855297, 15749284.83725323, 34540470.44772812],
       [100245293.71013911, 143676882.78436548, 167637019.7783451, ...,
        7825859.198666342, 34540470.44772814, 111012170.72205015]],
      dtype=object)

# Prediction

In [13]:
USER = 1066   # User to check
N = 10

## Ratings Prediction

### Get list of rated items

In [52]:
def find_item_neighbors(user, item, rating_matrix, similarity_matrix, K):
    corr_items_similarity = similarity_matrix[item]
    
    user_rated_items = rating_matrix[user]
    rated_item_indexes = []
    
    for item_index in range(len(user_rated_items)):
        
        # User has rated the item
        if user_rated_items[item_index] != 0:
            rated_item_indexes.append(item_index)
    
    filtered_similarity_scores = []
    
    # Get neighbors similarity score
    for item_index in rated_item_indexes:
        if item != item_index:
            filtered_similarity_scores.append(
                (item_index, corr_items_similarity[item_index])
            )
    
    # Sort based on simlarity scores 
    # (tuple is in (item_index, sim_score) format)
    filtered_similarity_scores = sorted(filtered_similarity_scores, 
                                       key=lambda x: x[1], 
                                       reverse=True
                                    )
    
    item_neighbors = []

    if len(filtered_similarity_scores) < K:
        item_neighbors = [sim_item[0] for sim_item in filtered_similarity_scores]
        for i in range(K - len(filtered_similarity_scores)):
            item_neighbors.append(-999)
    
    else:
        # Filter top K similar items
        for i in range(K):
            similar_item = filtered_similarity_scores[i]
            item_neighbors.append(similar_item[0])
        
    return item_neighbors

### Item-KNN

In [51]:
def item_KNN(USER, rating_matrix, user_item_context_encodings, M):
    wanted_user = user_item_context_encodings[0][USER]

    inferred_ratings = []
    user_rated_items = rating_matrix[wanted_user]

    for item in range(len(user_rated_items)):
        rating = user_rated_items[item]
        
        # Item has not been rated
        if rating == 0:
            rating_sum = 0
            neighbor_count = 0
            
            # Find k nearest item neighbors
            nearest_neighbors = find_item_neighbors(
                                    wanted_user,
                                    item,
                                    rating_matrix,
                                    M,
                                    K
                                )
            
            # Nearest neighbors are represented as their indexes
            # in the rating matrix
            for item_neighbor in nearest_neighbors:
                if item_neighbor != -999:
                    neighbor_rating = rating_matrix[wanted_user][item_neighbor]
                    
                    rating_sum += neighbor_rating
                    neighbor_count += 1
                
            inferred_rating = rating_sum / neighbor_count if neighbor_count != 0 else 0
            
            if inferred_rating != 0:
                inferred_ratings.append((item, inferred_rating))
    return inferred_ratings, user_rated_items

In [16]:
def get_mapped_ratings_from_context(inferred_ratings, user_item_context_reverse_encodings, context_T):
    mapped_ratings = []
    for i in inferred_ratings:
        
        # Get actual item encoding (from the cartesian product result)
        item_context = context_T[i[0]]
        
        item = user_item_context_reverse_encodings[1][item_context[0]]
        
        new_entry = [item]
        
        for j in range(1, len(item_context)):
            new_entry.append(item_context[j])
        
        new_entry.append(i[1])
        
        mapped_ratings.append(new_entry)
        
    return mapped_ratings

In [17]:

def get_predictions(mapped_ratings, encoded_df):
    predicted_df = pd.DataFrame(
        mapped_ratings, 
        columns=[
            'Item', 
            *encoded_df.columns[3:], 
            'predicted_rating']
    ).sort_values(by='predicted_rating', ascending=False)
    return predicted_df

## Recommendation Generation

In [18]:
def recommendation_generation(N, predicted_df):
    # Select N items to be recommended
    chosen = []

    for data in predicted_df.iterrows():
        data_tup = tuple(data[1])
        
        if len(chosen) <= N:
            chosen.append(data_tup)

    return chosen

## Rating Prediction Translated Result

In [19]:
def get_mapped_ratings_translated(inferred_ratings, context_T, user_item_context_reverse_encodings):
    mapped_ratings = []
    for i in inferred_ratings:
        
        # Get actual item encoding (from the cartesian product result)
        item_context = context_T[i[0]]
        
        item = user_item_context_reverse_encodings[1][item_context[0]]
        
        new_entry = [item]
        
        # Translate remaining context
        for j in range(1, len(item_context)):
            # 1 --> item
            # >= 2 --> context
            translator = user_item_context_reverse_encodings[j+1]
            new_entry.append(translator[item_context[j]])
        
        # Append the rating
        new_entry.append(i[1]*maximum_rating)
        
        mapped_ratings.append(new_entry)
    
    return mapped_ratings

In [20]:
def get_result(mapped_ratings, encoded_df):   
    res = pd.DataFrame(
        mapped_ratings, 
        columns=[
            'Item', 
            *encoded_df.columns[3:], 
            'predicted_rating']
    ).sort_values(by='predicted_rating', ascending=False)
    return res

## Dataset Merge

In [21]:
witheld = pd.read_csv('witheld_ratings.csv')

In [22]:
display(witheld)

,userid,itemid,rating,Time,Location,Companion
0,1066,tt4411490,1,Weekend,Cinema,Family
1,1066,tt1707386,2,Weekday,Home,Partner
2,1066,tt4411490,1,Weekday,Cinema,Family
3,1066,tt0232500,1,Weekday,Home,Partner
4,1066,tt1707386,1,Weekend,Cinema,Family
...,...,...,...,...,...,...
147,1098,tt0405422,3,Weekend,Cinema,Alone
148,1098,tt0211915,4,Weekend,Home,Family
149,1098,tt0169547,4,Weekend,Cinema,Alone
150,1098,tt0289879,1,Weekday,Home,Alone


In [23]:
def merge_df(res, witheld, USER):
    actual_ratings = []
    for row in res.iterrows():
        row_data = row[1]
        actual = witheld[
            (witheld['userid'] == USER) &\
            (witheld['itemid'] == row_data['Item']) &\
            (witheld['Time'] == row_data['Time']) &\
            (witheld['Location'] == row_data['Location']) &
            (witheld['Companion'] == row_data['Companion'])
        ]['rating']
        
        if(not actual.empty):
            actual_ratings.append(tuple(actual)[0])
            
        else:
            actual_ratings.append(np.nan)   
    

    return actual_ratings  

In [24]:
print(user_item_context_encodings[0])

{1001: 0, 1002: 1, 1003: 2, 1004: 3, 1005: 4, 1006: 5, 1007: 6, 1008: 7, 1009: 8, 1011: 9, 1014: 10, 1015: 11, 1016: 12, 1018: 13, 1026: 14, 1027: 15, 1028: 16, 1029: 17, 1030: 18, 1031: 19, 1032: 20, 1033: 21, 1034: 22, 1035: 23, 1037: 24, 1038: 25, 1039: 26, 1040: 27, 1041: 28, 1042: 29, 1043: 30, 1044: 31, 1045: 32, 1046: 33, 1047: 34, 1048: 35, 1049: 36, 1050: 37, 1051: 38, 1052: 39, 1053: 40, 1054: 41, 1055: 42, 1056: 43, 1057: 44, 1058: 45, 1059: 46, 1060: 47, 1061: 48, 1062: 49, 1063: 50, 1064: 51, 1065: 52, 1066: 53, 1067: 54, 1068: 55, 1069: 56, 1070: 57, 1071: 58, 1074: 59, 1075: 60, 1076: 61, 1077: 62, 1078: 63, 1079: 64, 1082: 65, 1084: 66, 1087: 67, 1097: 68, 1098: 69, 1105: 70, 1107: 71, 1109: 72, 1112: 73, 1113: 74, 1114: 75, 1115: 76, 1116: 77, 1119: 78, 1120: 79, 1122: 80}


# Automated User (Sum of RMSE)

In [28]:
from sklearn.metrics import mean_squared_error

In [53]:

sum_rms = 0
actual_users = 0

for USER in user_item_context_encodings[0]:
    N = 10
    inferred_ratings, user_rated_items = item_KNN(USER, rating_matrix, user_item_context_encodings, M)

    mapped_ratings = get_mapped_ratings_from_context(inferred_ratings, user_item_context_reverse_encodings, context_T)
    predicted_df = get_predictions(mapped_ratings, encoded_df)

    chosen = recommendation_generation(N, predicted_df)
    
    mapped_ratings = get_mapped_ratings_translated(inferred_ratings, context_T, user_item_context_reverse_encodings)
    
    res = get_result(mapped_ratings, encoded_df)
    
    actual_ratings = merge_df(res, witheld, USER)
    
    merged_result = res.copy()
    merged_result['actual_rating'] = actual_ratings
    merged_result[(merged_result['actual_rating'] != np.nan)]  
    merged_result.dropna(inplace=True)


    if len(merged_result) != 0:
        actual_users += 1
        rms = mean_squared_error(merged_result['actual_rating'], merged_result['predicted_rating'], squared=False)
        sum_rms += rms


print("actual users: " + str(actual_users))
print(sum_rms / actual_users)
    


actual users: 17
1.2582894802499536


# Automated User (Concatenate User DF)

In [ ]:

sum_rms = 0
actual_users = 0

combined_users = []

for USER in user_item_context_encodings[0]:
    N = 10
    K_similar_users, wanted_user = rating_prediction(USER)
    user_rated_items = get_rated_items(wanted_user)

    inferred_ratings = KNN(user_rated_items, K_similar_users)
    mapped_ratings = get_mapped_ratings_from_context(inferred_ratings, user_item_context_reverse_encodings, context_T)
    predicted_df = get_predictions(mapped_ratings, encoded_df)

    chosen = recommendation_generation(N, predicted_df)
    
    mapped_ratings = get_mapped_ratings_translated(inferred_ratings, context_T, user_item_context_reverse_encodings)
    
    res = get_result(mapped_ratings, encoded_df)
    
    actual_ratings = merge_df(res, witheld, USER)
    
    merged_result = res.copy()
    merged_result['actual_rating'] = actual_ratings
    merged_result[(merged_result['actual_rating'] != np.nan)]  
    merged_result.dropna(inplace=True)

    if len(merged_result) != 0:
        combined_users.append(merged_result)


df_final = pd.concat(combined_users)
print(mean_squared_error(df_final['actual_rating'], df_final['predicted_rating'], squared=False))

